In [65]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd
import pandas.util.testing as tm

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

import joypy

# Metadata files 

In [66]:
# gencode 25 metadata 
metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')

# gencode 35 metadata 
metadata_pc_g35 = pd.read_csv('tmp_res/metadata_pc_g35.txt', sep='\t')

# open metadata with scores and metrics
# meta = pd.read_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_refseqovlp_FRAMES_PhyloCSF.txt', sep='\t')

In [67]:
ups = 'GCACATGATACCACAACTACACAGATCTCAACAA'
dws = 'ACAATGGGCTAGCGTAACACAGACTATGAAC'

RS1 = 'AGATCT'
RS2 = 'GCTAGC'

print (len(ups), len(dws))

34 31


In [68]:
ATC_rich_full = 'CTAATATAAAAATTTAACTCATCTTCTTCTTATTTATAACCCACAACCACACAACAATACCATCCACACTAACTTCCTATAAATTCCTATACACATAAAATCCATTTTCCTATCTCTCACTTCCCACCTTCCACCATTAAACATAATAATTCCATTAATACACTTCCTTACCTTAACATCAAATCTAATTAAATCTCTACTAAATTTACCCTTCACTCTACTCCA'


# ATC_rich_full

len(ATC_rich_full)



225

In [69]:
pset = pd.read_csv('tmp_res/SET1.txt', sep='\t')
print (pset.tr_id.nunique(), pset.gene.nunique())

rset = pd.read_csv('tmp_res/SET2.txt', sep='\t')
print (rset.gene.nunique(), rset.tr_id.nunique())

85 60
393 395


In [70]:
rset[rset['Riboseq_Summary'].str.contains('ATG')]

,tr_id,gene,N_term_start1,N_term_end1,len_codons,global_coo_50_and_less,strand,global_coo_primary,PhyloCSF120score,number_of_records,ovlp,Riboseq_Summary,Coverage_value_ext,Proteomics_count_ext,tag,transcript_type,CDS_ratio
95,ENST00000467087.5,STIM2,258,528,90.0,chr4:26861069-26861218,+,chr4:26860949-26861218,-953.1598,64,0;0;0,ATG; 268-529; Rank: 462; cov: 73.17,73.170732,0.0,"non_ATG_start,basic,appris_principal_3,CCDS",protein_coding,0.139053
119,ENST00000316788.11,AP3S1,251,557,102.0,chr5:115841888-115842037,+,chr5:115841732-115842037,-1017.9102,94,0;0;0,ATG; 258-558; Rank: 121; cov: 68.42,68.421053,0.0,"basic,appris_principal_1,CCDS",protein_coding,0.325754
218,ENST00000418331.6,PTPRJ,1,352,117.0,chr11:47980763-47980912,+,chr11:47980562-47980912,-342.5714,48,0;0;0,ATG; 11-353; Rank: 369; cov: 60.55,60.550459,2.0,"basic,appris_principal_2,CCDS",protein_coding,0.407875


In [71]:
# open metadata with scores and metrics and overlaps 
meta = pd.read_csv('tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t')

# Flanks 

In [72]:
s = 'CGGACTCAGCGCTCGAGCCGGCTTGTCGACGACGGCGGTCTCCGTCGTCAGGATCATCCGCAcATTATACCaCAACTACAC'
s1 = 'ACCATGGGTaAAACAGACTTTaAATTTcGACCTTCTcAAGCTcGCGGGAGACGTCGAGTCCAACCCCGGGCC'

flank_right = s[-15:]

flank_left = s1[0:15]

# Make test casettes 


Split theoretical extensions (multiple by 3) by length, 75 codons = 225nt: 
* short ntes < 225nt 
* ntes == 225nt 
* long ntes > 225nt 


ntes 225nt long 

RS1 [-----75 codons-------] RS2

ntes shorter 225 nt 

RS1 [ATC-rich: (75 - X) codons] [----X codons----] RS2



ntes longer than 225nt -> split into overlpping chunks 

    [------------100 codons------------]

RS1 [----75 codons-----] RS2

                RS1 [----75 codons-----] RS2



In [73]:
225 / 3

75.0

In [74]:
short_ntes = {}
long_ntes = {}
ntes_225 = {}

for el in meta[meta['tr_id'].isin(pset.tr_id.tolist()+rset.tr_id.tolist())][['tr_id', 'N_term_start1', 'N_term_end1', 'transcript_seq']].to_numpy():
    tr_id = el[0]
    nt_start = el[1]
    nt_end = el[2]
    tr_seq = el[3]
    nte_seq = tr_seq[nt_start:nt_end]
    
    # add flag:
    if (tr_id in pset.tr_id.tolist()) & (tr_id in rset.tr_id.tolist()):
        flag = 'PhyloSET,RiboSET'
    elif (tr_id in pset.tr_id.tolist()) & (tr_id not in rset.tr_id.tolist()):
        flag = 'PhyloSET'
    else:
        flag = 'RiboSET'
    
    ##### th.TNE < 225 ####
    if len(nte_seq) < 225:
        
        short_ntes[tr_id] = {}
        short_ntes[tr_id]['nte_seq'] = nte_seq
        short_ntes[tr_id]['intervals'] = []
        
        # add ATC-rich sequence in front to make it 225 
        k = 225 - len(nte_seq)
        ATC_rich = ATC_rich_full[-k:]
        test_cassette_ = ATC_rich + '_' + nte_seq
        test_cassette = ATC_rich + nte_seq
        
        RS_sites_flag = []
        
        # check for restriction sites 
        if (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) == -1):
            # no RS sites
            RS_sites_flag = 'None;None'
        elif (test_cassette.find(RS1) != -1) & (test_cassette.find(RS2) == -1):
            RS_sites_flag = str(test_cassette.find(RS1))+';None'
        elif (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) != -1):
            RS_sites_flag = 'None;'+str(test_cassette.find(RS2))
        else:
            RS_sites_flag = str(test_cassette.find(RS1))+';'+ str(test_cassette.find(RS2))
            
            
        short_ntes[tr_id]['intervals'].append([test_cassette, test_cassette_, [k], RS_sites_flag, 'short', flag])
    
    elif len(nte_seq) == 225:
        test_cassette = nte_seq
        test_cassette_ = nte_seq
        
        ntes_225[tr_id] = {}
        ntes_225[tr_id]['nte_seq'] = nte_seq
        ntes_225[tr_id]['intervals'] = []
        
        # check for restriction sites 
        if (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) == -1):
            # no RS sites
            RS_sites_flag = 'None;None'
        elif (test_cassette.find(RS1) != -1) & (test_cassette.find(RS2) == -1):
            RS_sites_flag = str(test_cassette.find(RS1))+';None'
        elif (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) != -1):
            RS_sites_flag = 'None;'+str(test_cassette.find(RS2))
        else:
            RS_sites_flag = str(test_cassette.find(RS1))+';'+ str(test_cassette.find(RS2))
            
        
        ntes_225[tr_id]['intervals'].append([test_cassette, test_cassette_, [225], RS_sites_flag, 'exact', flag])
        
    else:
        long_ntes[tr_id] = {}
        long_ntes[tr_id]['nte_seq'] = nte_seq
        long_ntes[tr_id]['intervals'] = []
        
        # long ones = split them. Keep reading frame (all ovlps are multiple of 3)
        n = 1
        A = 0 #end of 1st interval 
        while len(nte_seq) - A > 195:
                
            interval = [225*(n-1) - 30*(n-1), 225*n - 30*(n-1)]
  
            # sequence
            test_cassette = nte_seq[interval[0]:interval[1]]
            
            if (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) == -1):
            # no RS sites
                RS_sites_flag = 'None;None'
            elif (test_cassette.find(RS1) != -1) & (test_cassette.find(RS2) == -1):
                RS_sites_flag = str(test_cassette.find(RS1))+';None'
            elif (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) != -1):
                RS_sites_flag = 'None;'+str(test_cassette.find(RS2))
            else:
                RS_sites_flag = str(test_cassette.find(RS1))+';'+ str(test_cassette.find(RS2))
        
        
            long_ntes[tr_id]['intervals'].append([test_cassette, test_cassette, interval, RS_sites_flag, 'long', flag])
            
            
            A = 225*n - 30*(n-1)
            n += 1
        else:
            # make the last interval 
            interval = [len(nte_seq) - 225, len(nte_seq)]
            test_cassette = nte_seq[interval[0]:interval[1]]
            
            if (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) == -1):
            # no RS sites
                RS_sites_flag = 'None;None'
            elif (test_cassette.find(RS1) != -1) & (test_cassette.find(RS2) == -1):
                RS_sites_flag = str(test_cassette.find(RS1))+';None'
            elif (test_cassette.find(RS1) == -1) & (test_cassette.find(RS2) != -1):
                RS_sites_flag = 'None;'+str(test_cassette.find(RS2))
            else:
                RS_sites_flag = str(test_cassette.find(RS1))+';'+ str(test_cassette.find(RS2))
            
            long_ntes[tr_id]['intervals'].append([test_cassette, test_cassette, interval, RS_sites_flag, 'long', flag])

In [75]:
short_ntes

{'ENST00000379389.4': {'nte_seq': 'GCAGGCTCGGCGGCACGCCCCCTGACGTGTGTGCCTCAGGCTTATAATAGGGCCGGTGCTGCCTGCCGAAGCCGGCGGCTGAGAGGCAGCGAACTCATCTTTGCCAGTACAGGAGCTTGTGCCGTGGCCCACAGCCCACAGCCCACAGCC',
  'intervals': [['TCCATTAATACACTTCCTTACCTTAACATCAAATCTAATTAAATCTCTACTAAATTTACCCTTCACTCTACTCCAGCAGGCTCGGCGGCACGCCCCCTGACGTGTGTGCCTCAGGCTTATAATAGGGCCGGTGCTGCCTGCCGAAGCCGGCGGCTGAGAGGCAGCGAACTCATCTTTGCCAGTACAGGAGCTTGTGCCGTGGCCCACAGCCCACAGCCCACAGCC',
    'TCCATTAATACACTTCCTTACCTTAACATCAAATCTAATTAAATCTCTACTAAATTTACCCTTCACTCTACTCCA_GCAGGCTCGGCGGCACGCCCCCTGACGTGTGTGCCTCAGGCTTATAATAGGGCCGGTGCTGCCTGCCGAAGCCGGCGGCTGAGAGGCAGCGAACTCATCTTTGCCAGTACAGGAGCTTGTGCCGTGGCCCACAGCCCACAGCCCACAGCC',
    [75],
    'None;None',
    'short',
    'RiboSET']]},
 'ENST00000349431.10': {'nte_seq': 'GCTGGTTCCGCCCCGCGAGCGGCCATCTTGGAGGCTGAGGCGGCGGCGGCGGCGCTGCGGCGGGTTCGGTGGGCCCAATCCCGGGGCGGTGCGGCTGTTTCGGGCGCGGGCCCCGCTTTTCCGCACCCTGCTCCGGCCTCGACTACGGCGAGCCTGAGCGCGGCGGCGGCCCACGCGCAGCGACAGGGAGAG',
  'intervals': [['ATCTCTACTAAATTTACCCTTCACT

In [76]:
len('GCAGGCTCGGCGGCACGCCCCCTGACGTGTGTGCCTCAGGCTTATAATAGGGCCGGTGCTGCCTGCCGAAGCCGGCGGCTGAGAGGCAGCGAACTCATCTTTGCCAGTACAGGAGCTTGTGCCGTGGCCCACAGCCCACAGCCCACAGCC')

150

In [77]:
len('TCCATTAATACACTTCCTTACCTTAACATCAAATCTAATTAAATCTCTACTAAATTTACCCTTCACTCTACTCCAGCAGGCTCGGCGGCACGCCCCCTGACGTGTGTGCCTCAGGCTTATAATAGGGCCGGTGCTGCCTGCCGAAGCCGGCGGCTGAGAGGCAGCGAACTCATCTTTGCCAGTACAGGAGCTTGTGCCGTGGCCCACAGCCCACAGCCCACAGCC')

225

In [78]:
long_ntes

{'ENST00000338370.7': {'nte_seq': 'AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGCGCTGCGGCAGGCGGGCCCGGGGTCCAGCCGAGAGGGTCCGGGCGCCAGGCAACGCGATTCGCGCGGGGGTGAACCCGGGGAGGGGGCCGGCCTCCCCGTTCTGGGACCTTTCGGTGCGTCCGAGCGCGACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGAAGCAAAGTCCTCCTTGTGGGTTGGGGTGGCGGAGGGAGAAGGGAAGCGAGGGTCGCGGCGGGACCAGACGCCCCAGTCCCGGCCCGCCCGCGACTACTGAAGGCGCTGCCGCCTGACCTGAACGGGCACTTGTGTTCCAGCTCCCCTGGGACCTGTGGCCGCCGCCCACAGACC',
  'intervals': [['AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGCGCTGCGGCAGGCGGGCCCGGGGTCCAGCCGAGAGGGTCCGGGCGCCAGGCAACGCGATTCGCGCGGGGGTGAACCCGGGGAGGGGGCCGGCCTCCCCGTTCTGGGACCTTTCGGTGCGTCCGAGCGCGACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGA',
    'AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGCGCTGCGGCAGGCGGGCCCGGGGTCCAGCCGAGAGGGTCCGGGCGCCAGGCAACGCGATTCGCGCGGGGGTGAACCCGGGGAGGGGGCCGGCCTCCCCGTTCTGGGACCTTTCGGTGCGTCCGAGCGCGACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGA',
    [0, 225],
    'None;None',
    'long',
    'RiboSET'],
   ['ACACCGCCCGCGTCGGGGGC

In [79]:
len('AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGCGCTGCGGCAGGCGGGCCCGGGGTCCAGCCGAGAGGGTCCGGGCGCCAGGCAACGCGATTCGCGCGGGGGTGAACCCGGGGAGGGGGCCGGCCTCCCCGTTCTGGGACCTTTCGGTGCGTCCGAGCGCGACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGAAGCAAAGTCCTCCTTGTGGGTTGGGGTGGCGGAGGGAGAAGGGAAGCGAGGGTCGCGGCGGGACCAGACGCCCCAGTCCCGGCCCGCCCGCGACTACTGAAGGCGCTGCCGCCTGACCTGAACGGGCACTTGTGTTCCAGCTCCCCTGGGACCTGTGGCCGCCGCCCACAGACC')

399

In [80]:
len('AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGCGCTGCGGCAGGCGGGCCCGGGGTCCAGCCGAGAGGGTCCGGGCGCCAGGCAACGCGATTCGCGCGGGGGTGAACCCGGGGAGGGGGCCGGCCTCCCCGTTCTGGGACCTTTCGGTGCGTCCGAGCGCGACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGA')

225

In [81]:
len('ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTGTTGGAAGCAAAGTCCTCCTTGTGGGTTGGGGTGGCGGAGGGAGAAGGGAAGCGAGGGTCGCGGCGGGACCAGACGCCCCAGTCCCGGCCCGCCCGCGACTACTGAAGGCGCTGCCGCCTGACCTGAACGGGCACTTGTGTTCCAGCTCCCCTGGGACCTGTGGCCGCCGCCCACAGACC')

225

# transform it to df 

In [82]:
li = []

for tr_id in long_ntes:
    
    print (tr_id, 'long')
    
    subdict = long_ntes[tr_id]
    nte_seq = subdict['nte_seq']
    intervals_li = subdict['intervals']
    
    for interval in intervals_li:
        test_cassette_seq = interval[0]
        test_cassette_seq_ = interval[1]
        coo = str(interval[2])
        RS = interval[3]
        type_ = interval[4]
        flag = interval[5]

        len_t_c = len(test_cassette)
        li.append([tr_id, nte_seq, test_cassette_seq, test_cassette_seq_, coo, RS, type_, len_t_c, flag])
        
        
for tr_id in short_ntes:
    
    print (tr_id, 'short')
    
    subdict = short_ntes[tr_id]
    
    nte_seq = subdict['nte_seq']
    intervals_li = subdict['intervals']
    
    for interval in intervals_li:
        test_cassette_seq = interval[0]
        test_cassette_seq_ = interval[1]
        coo = str(interval[2])
        RS = interval[3]
        type_ = interval[4]
        flag = interval[5]

        len_t_c = len(test_cassette)
        li.append([tr_id, nte_seq, test_cassette_seq, test_cassette_seq_, coo, RS, type_, len_t_c, flag])
        
        
for tr_id in ntes_225:
    
    print (tr_id, '225')
    
    subdict = ntes_225[tr_id]
    
    nte_seq = subdict['nte_seq']
    intervals_li = subdict['intervals']
    
    for interval in intervals_li:
        test_cassette_seq = interval[0]
        test_cassette_seq_ = interval[1]
        coo = str(interval[2])
        RS = interval[3]
        type_ = interval[4]
        flag = interval[5]

        len_t_c = len(test_cassette)
        li.append([tr_id, nte_seq, test_cassette_seq, test_cassette_seq_, coo, RS, type_, len_t_c, flag])

ENST00000338370.7 long
ENST00000455833.6 long
ENST00000302118.5 long
ENST00000370544.9 long
ENST00000370238.7 long
ENST00000370035.7 long
ENST00000369799.9 long
ENST00000368811.7 long
ENST00000357325.9 long
ENST00000368061.2 long
ENST00000258341.4 long
ENST00000259154.8 long
ENST00000302550.15 long
ENST00000234091.8 long
ENST00000234115.10 long
ENST00000234160.4 long
ENST00000354785.8 long
ENST00000398888.6 long
ENST00000305533.9 long
ENST00000356692.9 long
ENST00000295633.7 long
ENST00000462833.5 long
ENST00000333762.5 long
ENST00000476941.5 long
ENST00000273482.10 long
ENST00000325404.2 long
ENST00000392456.3 long
ENST00000337190.6 long
ENST00000467087.5 long
ENST00000359707.8 long
ENST00000296504.3 long
ENST00000166345.7 long
ENST00000382038.6 long
ENST00000439211.6 long
ENST00000379982.7 long
ENST00000261483.4 long
ENST00000316788.11 long
ENST00000296930.9 long
ENST00000259808.8 long
ENST00000211379.9 long
ENST00000229794.8 long
ENST00000355773.5 long
ENST00000369750.3 long
ENST000

In [83]:
pset_test_cassettes = pd.DataFrame(li, columns = ['tr_id', 'nte_seq', 'test_cassette_seq', 
                           'test_cassette_seq_', 'coo', 'RS', 'type',
                           'test_cassette_len', 'set']).merge(metadata_pc_g25[['tr_id', 'gene']],
                                                             on='tr_id', how='inner')



print (pset_test_cassettes.shape[0])

610


In [84]:
pset_test_cassettes['tr_id'].nunique()

472

In [85]:
395+85-8  # 3 AUGs in rset; 60 genes and 85 transcrtips in phyloset - some redundancy as well => can remove identical NTEs 

472

In [86]:
pset.tr_id.nunique(), rset.tr_id.nunique()

(85, 395)

In [87]:
len(set(pset.tr_id.tolist()).intersection(rset.tr_id.tolist()))

8

In [88]:
pset_test_cassettes[0:2]

,tr_id,nte_seq,test_cassette_seq,test_cassette_seq_,coo,RS,type,test_cassette_len,set,gene
0,ENST00000338370.7,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,"[0, 225]",None;None,long,225,RiboSET,AURKAIP1
1,ENST00000338370.7,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,"[174, 399]",None;None,long,225,RiboSET,AURKAIP1


# Codon to AA 

In [89]:
table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }

reverse_t = defaultdict(list)

for codon, aa in table.items():
    if aa not in reverse_t:
        reverse_t[aa] = [codon]
    else:
        reverse_t[aa].append(codon)
        
reverse_t

defaultdict(list,
            {'I': ['ATA', 'ATC', 'ATT'],
             'M': ['ATG'],
             'T': ['ACA', 'ACC', 'ACG', 'ACT'],
             'N': ['AAC', 'AAT'],
             'K': ['AAA', 'AAG'],
             'S': ['AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'],
             'R': ['AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'],
             'L': ['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
             'P': ['CCA', 'CCC', 'CCG', 'CCT'],
             'H': ['CAC', 'CAT'],
             'Q': ['CAA', 'CAG'],
             'V': ['GTA', 'GTC', 'GTG', 'GTT'],
             'A': ['GCA', 'GCC', 'GCG', 'GCT'],
             'D': ['GAC', 'GAT'],
             'E': ['GAA', 'GAG'],
             'G': ['GGA', 'GGC', 'GGG', 'GGT'],
             'F': ['TTC', 'TTT'],
             'Y': ['TAC', 'TAT'],
             '_': ['TAA', 'TAG', 'TGA'],
             'C': ['TGC', 'TGT'],
             'W': ['TGG']})

## Change restriction site codons to synonymous codons 

In [90]:
#  AGATCT CAACAAGCCACC ATG GG GCTAGC 

In [91]:
def get_changed_sequence(RS_seq, orf_seq, orf_seq_aa):
    
    orf_seq_aa = orf_seq_aa[1:-1] #remove start and stop
    
    
    # search for restriction site in extension sequence 
    if orf_seq.find(RS_seq) != -1:
        pos_start = orf_seq.find(RS_seq)
        pos_end = pos_start + 6
        
        # codons in the right frame?? 
        codon_coo = zip([i for i in range(0, int(len(orf_seq)/3))], [np.arange(i, i+3) for i in list(range(0, len(orf_seq), 3))])
        
        #affected_codon_number_start = None 
        
        for codon_number, codon_coo_li in dict(codon_coo).items():
            if pos_start in codon_coo_li:
                affected_codon_number_start = codon_number
                prev_codon_number = affected_codon_number_start
                prev_position = prev_codon_number * 3
                
            if pos_end in codon_coo_li:
                affected_codon_number_stop = codon_number
                next_codon_number = affected_codon_number_stop+1
                next_position = next_codon_number * 3
                
        #print ('affected_codon_number_start:', affected_codon_number_start)
        #print ('affected_codon_number_stop:', affected_codon_number_stop)
                
        # change 2nd codon (its either 2 codons or 3 affected by restriction site)
        change_codon_pos = (affected_codon_number_start+1)*3 # position of codon (in sequence coo)
        codon_seq_before = orf_seq[change_codon_pos:change_codon_pos+3]
        aa_seq_before = table[codon_seq_before] 
        
        # choose synonymous if any 
        if len(reverse_t[aa_seq_before]) > 1:
            for el in reverse_t[aa_seq_before]:
                if el != codon_seq_before:
                    # story about serine 
                    if codon_seq_before == 'AGC':
                        #print ('Ser AGC')
                        codon_seq_subset = 'AGT'
                    elif codon_seq_before == 'AGT':
                        #print ('Ser AGT')
                        codon_seq_subset = 'AGC'
                    elif (codon_seq_before.startswith('TC')) & (el.startswith('TC')):
                        #print ('Ser TC_X')
                        codon_seq_subset = el
                    else:     
                        codon_seq_subset = el
                    
                    # change in the sequence
                    NEW_ORF_SEQ = orf_seq[0:change_codon_pos]+codon_seq_subset+orf_seq[change_codon_pos+3:]
                    
        else:
            #choose 1st codon to change 
            #print ('1st codon will be changed')
            change_codon_pos = affected_codon_number_start*3
            codon_seq_before = orf_seq[change_codon_pos:change_codon_pos+3]
            aa_seq_before = table[codon_seq_before] 
            
            if len(reverse_t[aa_seq_before]) > 1:
                for el in reverse_t[aa_seq_before]:
                    if el != codon_seq_before:
                        codon_seq_subset = el
                        
                        # change in the sequence
                        NEW_ORF_SEQ = orf_seq[0:change_codon_pos]+codon_seq_subset+orf_seq[change_codon_pos+3:]
            else:
                print ('lol kek cheburek')
                
        #print ('check: ', NEW_ORF_SEQ.find(bgi), orf_seq_aa == Seq(NEW_ORF_SEQ).translate())
        
        
    else: 
        NEW_ORF_SEQ = orf_seq
        
    return NEW_ORF_SEQ

In [92]:
# test_cassette_ has '_' between ATC-rich and extension 

In [93]:
pset_test_cassettes[0:2]

,tr_id,nte_seq,test_cassette_seq,test_cassette_seq_,coo,RS,type,test_cassette_len,set,gene
0,ENST00000338370.7,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,"[0, 225]",None;None,long,225,RiboSET,AURKAIP1
1,ENST00000338370.7,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,"[174, 399]",None;None,long,225,RiboSET,AURKAIP1


In [94]:
li = []

for el in pset_test_cassettes[['tr_id', 'gene', 'RS', 'test_cassette_seq', 'coo']].to_numpy():
    tr_id = el[0]
    gene = el[1]
    rs = el[2]
    test_cassette_seq_nt = el[3]
    test_cassette_seq_aa = str(Seq(test_cassette_seq_nt).translate())
    coo = el[-1]
    
    if rs != 'None;None':
        
        
        test_rs1 = get_changed_sequence(RS_seq='AGATCT', orf_seq=test_cassette_seq_nt, orf_seq_aa=test_cassette_seq_aa)
        
        test_rs1_aa = str(Seq(test_rs1).translate())
        
        test_rs2 = get_changed_sequence(RS_seq='GCTAGC', orf_seq=test_rs1, orf_seq_aa=test_rs1_aa)
        
        #print (tr_id, gene)
        
        #print (test_rs2 == test_cassette_seq_nt, test_rs2.find('AGATCT'), test_rs2.find('GCTAGC'))
        #print ('AGATCT', test_cassette_seq_nt.find('AGATCT'))
        #print ('GCTAGC', test_cassette_seq_nt.find('GCTAGC'))
        
        #print ()
        
        #print ('original: ')
        #print (test_cassette_seq_nt)
        
        #print ()
        #print ('after RS:')
        #print (test_rs2)
        #print ('####################')
        
        new_test_cassette = test_rs2
        mark = 'RS_removed'
        
    else:
        new_test_cassette = test_cassette_seq_nt 
        mark = '-'
        
    li.append([tr_id, gene, mark, new_test_cassette, coo])
    

    
rs_clean = pd.DataFrame(li, columns = ['tr_id', 'gene', 'mark', 'no_RS', 'coo'])

In [95]:
rs_clean.shape[0]

610

In [96]:
rs_clean[0:3]

,tr_id,gene,mark,no_RS,coo
0,ENST00000338370.7,AURKAIP1,-,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,"[0, 225]"
1,ENST00000338370.7,AURKAIP1,-,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,"[174, 399]"
2,ENST00000455833.6,IFFO2,-,CAGAAGGGGCAGTGCCGGCCGAGAGTCCTCTGCGTGCGGCTGCCGC...,"[0, 225]"


In [97]:
# ENST00000369622.7 SYNCRIP 

original = 'GAAGGGAAGGGGGGGTCACGCGGGGGCGCGCGCGCGCACCGGGAGCGCGCTCGGAGGCGAGTGGAACTGGATCGGGTTTGCTGCCAGCGGCGTGAGCTTCGGCCGCCATTTTACAACAGCTCCACTCGCGCCGGACACAGGGAGCAGCGAGCACGCGTTTCCCGCAACCCGATACCATCGGACAGGATTTCTCCGCCTCAGCCCAACGGGGAGATCTCTGGAAAC'

changed = 'GAAGGGAAGGGGGGGTCACGCGGGGGCGCGCGCGCGCACCGGGAGCGCGCTCGGAGGCGAGTGGAACTGGATCGGGTTTGCTGCCAGCGGCGTGAGCTTCGGCCGCCATTTTACAACAGCTCCACTCGCGCCGGACACAGGGAGCAGCGAGCACGCGTTTCCCGCAACCCGATACCATCGGACAGGATTTCTCCGCCTCAGCCCAACGGGGAGATTTCTGGAAAC'


print (original.find('AGATCT'))
print (changed.find('GCTAGC'), changed.find('AGATCT'))


codons_rs = [(i, changed[i:i+3]) for i in range(0, len(changed), 3)]
codons_before = [(i, original[i:i+3]) for i in range(0, len(original), 3)]

211
-1 -1


In [98]:
# GAG ATC T
   -- --- -

# codon before

(210, 'GAG'),
(213, 'ATC'),

# codons_rs

(210, 'GAG'),
(213, 'ATT')

IndentationError: unexpected indent (<ipython-input-98-f3a4d236b634>, line 2)

In [99]:
table['ATC'],table['ATT']

('I', 'I')

In [100]:
# ENST00000234091.8 ID2

original = 'AAGCGCCCGCTCGTCTTGATAGACGTGCCACCTTCCGCCAATGGGGACGAAGGGAAGCTCCAGCGTGTGGCCCCGGCGAGTGCGGATAAAAGCCGCCCCGCCGGGCTCGGGCTTCATTCTGAGCCGAGCCCGGTGCCAAGCGCAGCTAGCTCAGCAGGCGGCAGCGGCGGCCTGAGCTTCAGGGCAGCCAGCTCCCTCCCGGTCTCGCCTTCCCTCGCGGTCAGC'

changed = 'AAGCGCCCGCTCGTCTTGATAGACGTGCCACCTTCCGCCAATGGGGACGAAGGGAAGCTCCAGCGTGTGGCCCCGGCGAGTGCGGATAAAAGCCGCCCCGCCGGGCTCGGGCTTCATTCTGAGCCGAGCCCGGTGCCAAGCGCAGCTAGTTCAGCAGGCGGCAGCGGCGGCCTGAGCTTCAGGGCAGCCAGCTCCCTCCCGGTCTCGCCTTCCCTCGCGGTCAGC'

print (original.find('GCTAGC'))
print (changed.find('GCTAGC'), changed.find('AGATCT'))

codons_rs = [(i, changed[i:i+3]) for i in range(0, len(changed), 3)]
codons_before = [(i, original[i:i+3]) for i in range(0, len(original), 3)]

144
-1 -1


In [101]:
# original
(144, 'GCT'),
(147, 'AGT'),

# changed 
(144, 'GCT'),
(147, 'AGC'),

((147, 'AGC'),)

In [102]:
# ENST00000259154.8 KCTD3

original = 'GTCGGTGGCAGCGGAGCACGGAGAAGAGGCCCGGGCGGCCCGGCGGCCTGGAGGCCGCGAAAGGTGGAGGCCGGGCCGCCCTTGTGCACCGCAGGATTGACCCGGGAAGGGGCAGAAGCTAGCGAGCCCCGCCCGGCCGCCGGGAAGGTGGGGGAAGCCCCGTGCACCCCCCGCCCTCCGGCCGCCGCCGCCCCGCTGGCCCTGCAGCCGTCGCCGCTGCCTCGG'
afterrs = 'GTCGGTGGCAGCGGAGCACGGAGAAGAGGCCCGGGCGGCCCGGCGGCCTGGAGGCCGCGAAAGGTGGAGGCCGGGCCGCCCTTGTGCACCGCAGGATTGACCCGGGAAGGGGCAGAAGCTAGTGAGCCCCGCCCGGCCGCCGGGAAGGTGGGGGAAGCCCCGTGCACCCCCCGCCCTCCGGCCGCCGCCGCCCCGCTGGCCCTGCAGCCGTCGCCGCTGCCTCGG'

print (original.find('GCTAGC'))
print (afterrs.find('GCTAGC'), afterrs.find('AGATCT'))

codons_rs = [(i, afterrs[i:i+3]) for i in range(0, len(afterrs), 3)]
codons_before = [(i, original[i:i+3]) for i in range(0, len(original), 3)]

117
-1 -1


In [103]:
# original 

(117, 'GCT'),
(120, 'AGC'),

# after removal 

(117, 'GCT'),
(120, 'AGT'),

((120, 'AGT'),)

# Check reading frame for LONG extensions - is it the same 

In [104]:
pset_test_cassettes[pset_test_cassettes['type'] == 'long'].groupby(by='tr_id').count().sort_values(by='nte_seq')[-5:]

,nte_seq,test_cassette_seq,test_cassette_seq_,coo,RS,type,test_cassette_len,set,gene
tr_id,,,,,,,,,
ENST00000262067.4,3,3,3,3,3,3,3,3,3
ENST00000406869.5,3,3,3,3,3,3,3,3,3
ENST00000462833.5,3,3,3,3,3,3,3,3,3
ENST00000269228.9,3,3,3,3,3,3,3,3,3
ENST00000221930.5,4,4,4,4,4,4,4,4,4


In [105]:
for el in long_ntes['ENST00000221930.5']['intervals']:
    print (el[2])
    print (str(Seq(el[1]).translate()))
    
    # each ovlp is 10codons 
    # the last one ovlps is longer, 29 codons  

[0, 225]
AARGAGLGPERRKESPRSSLRPQSLRRAAAAPATAGRRGRRSGRRDELVGRRGKKLLRLFRCRWEPEARGPLGAT
[195, 420]
PEARGPLGATLPREEAGLGDPRPPPFAAGDACSLPAPYTASLRRPHSGPALGSRRPGLPQRLFPRPRAHPLHAAF
[390, 615]
PRAHPLHAAFIPGLSPEPPRILDPFSSRRRISLRPATDPLFKTTHLLVPDRAHLGYFRGILRHPRSKPPLHHCAL
[528, 753]
LVPDRAHLGYFRGILRHPRSKPPLHHCALLPEDLSFPSRPSYLLPGDPQPLQGRGLPTTPALFALSAVPGGAASP


In [106]:
str(Seq(long_ntes['ENST00000221930.5']['nte_seq']).translate())

'AARGAGLGPERRKESPRSSLRPQSLRRAAAAPATAGRRGRRSGRRDELVGRRGKKLLRLFRCRWEPEARGPLGATLPREEAGLGDPRPPPFAAGDACSLPAPYTASLRRPHSGPALGSRRPGLPQRLFPRPRAHPLHAAFIPGLSPEPPRILDPFSSRRRISLRPATDPLFKTTHLLVPDRAHLGYFRGILRHPRSKPPLHHCALLPEDLSFPSRPSYLLPGDPQPLQGRGLPTTPALFALSAVPGGAASP'

# Check in-frame AUGs in original NTEs and mark them in our list (at least 3 genes from RiboSET)

In [107]:
li = []

for el in pset_test_cassettes[['tr_id', 'nte_seq', 'gene']].to_numpy():
    nte_seq = el[1]
    tr_id = el[0]
    gene = el[-1]
    
    triplets = [nte_seq[i:i+3] for i in range(0, len(nte_seq), 3)]
    
    
    in_frame_ATG_info = ''
    for k,codon in enumerate(triplets):
        if codon == 'ATG':
            in_frame_ATG_info += 'ATG in %s position in ext; ' % str(k*3)
            
    #print (gene, in_frame_ATG_info) 
    
    li.append([gene, tr_id, in_frame_ATG_info])
    
df = pd.DataFrame(li, columns =['gene', 'tr_id', 'in_frame_ATG_in_theoretical_extension'])

df

,gene,tr_id,in_frame_ATG_in_theoretical_extension
0,AURKAIP1,ENST00000338370.7,
1,AURKAIP1,ENST00000338370.7,
2,IFFO2,ENST00000455833.6,
3,IFFO2,ENST00000455833.6,
4,PCSK9,ENST00000302118.5,
...,...,...,...
605,CD99,ENST00000381192.8_PAR_Y,
606,ZMYND19,ENST00000298585.2,
607,MRPS7,ENST00000245539.10,
608,TMX3,ENST00000299608.6,ATG in 18 position in ext;


In [108]:
pset_test_cassettes2 = pset_test_cassettes.merge(df, on=['tr_id', 'gene'], how='inner').drop_duplicates()

pset_test_cassettes3 = pset_test_cassettes2.merge(rs_clean, on=['tr_id', 'gene', 'coo'], how='inner').drop_duplicates()

print (pset_test_cassettes.shape[0], pset_test_cassettes2.shape[0], pset_test_cassettes3.shape[0], rs_clean.shape[0])

610 610 610 610


In [109]:
pre_final_df = pset_test_cassettes3[['tr_id', 'gene', 'nte_seq', 'coo', 'type', 'set', 'in_frame_ATG_in_theoretical_extension', 'mark', 'no_RS']]


pre_final_df.columns

Index(['tr_id', 'gene', 'nte_seq', 'coo', 'type', 'set',
       'in_frame_ATG_in_theoretical_extension', 'mark', 'no_RS'],
      dtype='object')

# Add controls: they are short => ATC-reach in front to make it 75 codons long 

In [110]:
75*2

150

In [111]:
ATG      AGATCT CAA CAA GCC ACC ATG GG GCTAGC
CTG      AGATCT CAA CAA GCC ACC ATG GG GCTAGC

CAA      AGATCT ACC CAA GG GCTAGC

POLG CTG AGATCT GTA AAA GAA GCC AAG CTG GAG GC GCTAGC

POLG ATG AGATCT GTA AAA GAA GCC AAG ATG GAG GC GCTAGC

POLG GTG AGATCT GTA AAA GAA GCC AGG TGG AG GC GCTAGC

POLG CGG AGATCT GTA AAA GAA GCC AAG CTG GAG GC GCTAGC

SyntaxError: invalid syntax (<ipython-input-111-cb39c9f3b3b1>, line 1)

In [112]:
len(ATC_rich_full)

225

In [113]:
flank_left = 'GCAcATTATACCaCAACTACAC'
flank_right = 'ACCATGGGTaAAACAGACTTTa'

In [114]:
# POSITIVE CONTROL WITH ATG 

li = []

atg_pos = 'CAACAAGCCACCATG'  # GG 
atc_rich_nt = (75 - int(len(atg_pos)/3))*3
atg_pos_225nt = ATC_rich_full[0:atc_rich_nt]+atg_pos
print (len(atg_pos_225nt))
atg_pos_225nt_final = flank_left + RS1 + atg_pos_225nt + 'GG' + RS2 + flank_right
li.append([atg_pos_225nt, atg_pos_225nt_final, 'positive_control_ATG_good_context'])

# CTG CONTROL 
ctg_pos = 'CAACAAGCCACCCTG'  # GG 
atc_rich_nt2 = (75 - int(len(ctg_pos)/3))*3
ctg_pos_225nt = ATC_rich_full[0:atc_rich_nt2]+ctg_pos
print (len(ctg_pos_225nt))
ctg_pos_225nt_final = flank_left + RS1 + ctg_pos_225nt + 'GG' + RS2 + flank_right
li.append([ctg_pos_225nt, ctg_pos_225nt_final, 'positive_control_CTG_good_context'])

# CAA control (negative)
caa_pos = 'ACCCAA'  # GG 
atc_rich_nt3 = (75 - int(len(caa_pos)/3))*3
caa_pos_225nt = ATC_rich_full[0:atc_rich_nt3]+caa_pos
print (len(caa_pos_225nt))
caa_pos_225nt_final = flank_left + RS1 + caa_pos_225nt + 'GG' + RS2 + flank_right
li.append([caa_pos_225nt, caa_pos_225nt_final, 'negative_control_CAA_good_context'])

# POLG CTG
polg_ctg_pos = 'GTAAAAGAAGCCAAGCTGGAG'  # GC 
atc_rich_nt4 = (75 - int(len(polg_ctg_pos)/3))*3
polg_ctg_pos_225nt = ATC_rich_full[0:atc_rich_nt4]+polg_ctg_pos
print (len(polg_ctg_pos_225nt))
polg_ctg_pos_225nt_final = flank_left + RS1 + polg_ctg_pos_225nt + 'GC' + RS2 + flank_right
li.append([polg_ctg_pos_225nt, polg_ctg_pos_225nt_final, 'POLG_CTG_good_context'])


# POLG ATG
polg_atg_pos = 'GTAAAAGAAGCCAAGATGGAG'  # GC 
atc_rich_nt5 = (75 - int(len(polg_atg_pos)/3))*3
polg_atg_pos_225nt = ATC_rich_full[0:atc_rich_nt5]+polg_atg_pos
print (len(polg_ctg_pos_225nt))
polg_atg_pos_225nt_final = flank_left + RS1 + polg_atg_pos_225nt + 'GC' + RS2 + flank_right
li.append([polg_atg_pos_225nt, polg_atg_pos_225nt_final, 'POLG_ATG_good_context'])


# POLG GTG
polg_gtg_pos = 'GTAAAAGAAGCCAAGGTGGAG'  # GC 
atc_rich_nt6 = (75 - int(len(polg_gtg_pos)/3))*3
polg_gtg_pos_225nt = ATC_rich_full[0:atc_rich_nt6]+polg_gtg_pos
print (len(polg_gtg_pos_225nt))
polg_gtg_pos_225nt_final = flank_left + RS1 + polg_gtg_pos_225nt + 'GC' + RS2 + flank_right
li.append([polg_gtg_pos_225nt, polg_gtg_pos_225nt_final, 'POLG_GTG_good_context'])


# POLG CGG 
polg_cgg_pos = 'GTAAAAGAAGCCAAGCGGGAG'  # GC 
atc_rich_nt7 = (75 - int(len(polg_cgg_pos)/3))*3
polg_cgg_pos_225nt = ATC_rich_full[0:atc_rich_nt7]+polg_cgg_pos
print (len(polg_cgg_pos_225nt))
polg_cgg_pos_225nt_final = flank_left + RS1 + polg_cgg_pos_225nt + 'GC' + RS2 + flank_right
li.append([polg_cgg_pos_225nt, polg_cgg_pos_225nt_final, 'POLG_CGG_good_context'])

225
225
225
225
225
225
225


In [115]:
CONTROLS_Martina_df = pd.DataFrame(li, columns = ['text_seq', 'full_cassette_with_RS_and_flanks_22nt', 'desc'])

In [116]:
CONTROLS_Martina_df.to_csv('data/controls_Martina_22nt_flanks.txt', sep='\t', index=False)

In [117]:
12+30+2+225

269

# My controls: positive = nonAUG CDSs 

In [118]:
flank_left = 'GCAcATTATACCaCAACTACAC'
flank_right = 'ACCATGGGTaAAACAGACTTTa'

In [119]:
RS1, RS2

('AGATCT', 'GCTAGC')

In [120]:
flank_left + RS1 + atg_pos_225nt + 'GG' + RS2 + flank_right

'GCAcATTATACCaCAACTACACAGATCTCTAATATAAAAATTTAACTCATCTTCTTCTTATTTATAACCCACAACCACACAACAATACCATCCACACTAACTTCCTATAAATTCCTATACACATAAAATCCATTTTCCTATCTCTCACTTCCCACCTTCCACCATTAAACATAATAATTCCATTAATACACTTCCTTACCTTAACATCAAATCTAATTAAATCTCTACTAAATTTACCCAACAAGCCACCATGGGGCTAGCACCATGGGTaAAACAGACTTTa'

### DYRK1B

In [121]:
metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains('ENST00000593685.5')].iloc[0]

tr_id                                              ENST00000593685.5
gene                                                          DYRK1B
gene_tr                                                   DYRK1B-006
transcript_seq     TTCCGGTCTCAGCAGGCCCAGGAGGGCCTCCTGGGTGGGGGGCGGG...
5UTR_start_seq     TTCCGGTCTCAGCAGGCCCAGGAGGGCCTCCTGGGTGGGGGGCGGG...
CDS_start                                                        ACC
cds_seq            ATGGCCGTCCCACCGGGCCATGGTCCCTTCTCTGGCTTCCCAGGGC...
cds_start_codon                                                  ATG
cds_stop_codon                                                   TGA
cds_start_pos                                                    469
cds_stop_pos                                                    2359
utr5_start                                                         0
utr5_end                                                         469
record_id          ENST00000593685.5|ENSG00000105204.13|OTTHUMG00...
Name: 82905, dtype: object

In [122]:
tr_seq = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains('ENST00000593685.5')].iloc[0].transcript_seq

leader5 = tr_seq[0:469]

leader5

'TTCCGGTCTCAGCAGGCCCAGGAGGGCCTCCTGGGTGGGGGGCGGGACGCCGGGTCCCTAGGGGCTGGTGGTCACTCAGGGTGGGGCGTGTCGCCCCTCCCCCGTCCACCTGCTCTACTCTTCCCCCGCGTGCCCTGGGCTGACCCTTGTCCCCTCCTCTCCCCGCCCCCGGTGGCAGTGGCGGCTGCTGTTGTCACCCACCGGGCCTCCTGTCCCGCTTGCCCTCCCCGCCGCGGGGCCGGCCGGGCCAGAGACAGGCGGTCGCCTTTTCAGCGCCGCCACCGCCGCCATGCTGGCCGCTCGCCCACCCCACTGGGGGCCCCACCGCGCCCCAGCCCCCCGTGGGCCCCGCGCCAGCCCTGACCCGGGTCTCAGCGGCGGTGGCAGCCGAGGTGCAGGATGCAAGAAGGCGCCCCCCGGCCGGGCTCCCGCTCCAGGCCTCGCTCCCCTGCGGCCCTCTGAGCCCACC'

In [123]:
# find (1) int(u)ORF and put it in-frame 

In [124]:
# N-term ext is already in-frame 

In [125]:
len(leader5[242:467])  #380 start in the middle 

225

In [64]:
leader5[380:390]

'GTGGCAGCCG'

In [127]:
s = leader5[242:467] 

' '.join([s[i:i+3] for i in range(0, len(s), 3)])

'CCG GGC CAG AGA CAG GCG GTC GCC TTT TCA GCG CCG CCA CCG CCG CCA TGC TGG CCG CTC GCC CAC CCC ACT GGG GGC CCC ACC GCG CCC CAG CCC CCC GTG GGC CCC GCG CCA GCC CTG ACC CGG GTC TCA GCG GCG GTG GCA GCC GAG GTG CAG GAT GCA AGA AGG CGC CCC CCG GCC GGG CTC CCG CTC CAG GCC TCG CTC CCC TGC GGC CCT CTG AGC CCA'

In [57]:
87/3, 380+87, 

(29.0, 467)

In [61]:
225-87, 380-138

(138, 242)

In [129]:
DYRK1B_GUG_uORF = flank_left + RS1 + s + 'GG' + RS2 + flank_right

len(DYRK1B_GUG_uORF)

283

In [137]:
DYRK1B_GUG_uORF

'GCAcATTATACCaCAACTACACAGATCTCCGGGCCAGAGACAGGCGGTCGCCTTTTCAGCGCCGCCACCGCCGCCATGCTGGCCGCTCGCCCACCCCACTGGGGGCCCCACCGCGCCCCAGCCCCCCGTGGGCCCCGCGCCAGCCCTGACCCGGGTCTCAGCGGCGGTGGCAGCCGAGGTGCAGGATGCAAGAAGGCGCCCCCCGGCCGGGCTCCCGCTCCAGGCCTCGCTCCCCTGCGGCCCTCTGAGCCCAGGGCTAGCACCATGGGTaAAACAGACTTTa'

In [130]:
len(s)

225

In [132]:
15+15+12+2+225

269

In [136]:
len(flank_left) + len(RS1) + len(flank_right) + len(RS2) + 225 + 2

283

In [ ]:
# NTE 

In [140]:
[leader5[1:][i:i+3] for i in range(0, len(leader5[1:]), 3)]

['TCC',
 'GGT',
 'CTC',
 'AGC',
 'AGG',
 'CCC',
 'AGG',
 'AGG',
 'GCC',
 'TCC',
 'TGG',
 'GTG',
 'GGG',
 'GGC',
 'GGG',
 'ACG',
 'CCG',
 'GGT',
 'CCC',
 'TAG',
 'GGG',
 'CTG',
 'GTG',
 'GTC',
 'ACT',
 'CAG',
 'GGT',
 'GGG',
 'GCG',
 'TGT',
 'CGC',
 'CCC',
 'TCC',
 'CCC',
 'GTC',
 'CAC',
 'CTG',
 'CTC',
 'TAC',
 'TCT',
 'TCC',
 'CCC',
 'GCG',
 'TGC',
 'CCT',
 'GGG',
 'CTG',
 'ACC',
 'CTT',
 'GTC',
 'CCC',
 'TCC',
 'TCT',
 'CCC',
 'CGC',
 'CCC',
 'CGG',
 'TGG',
 'CAG',
 'TGG',
 'CGG',
 'CTG',
 'CTG',
 'TTG',
 'TCA',
 'CCC',
 'ACC',
 'GGG',
 'CCT',
 'CCT',
 'GTC',
 'CCG',
 'CTT',
 'GCC',
 'CTC',
 'CCC',
 'GCC',
 'GCG',
 'GGG',
 'CCG',
 'GCC',
 'GGG',
 'CCA',
 'GAG',
 'ACA',
 'GGC',
 'GGT',
 'CGC',
 'CTT',
 'TTC',
 'AGC',
 'GCC',
 'GCC',
 'ACC',
 'GCC',
 'GCC',
 'ATG',
 'CTG',
 'GCC',
 'GCT',
 'CGC',
 'CCA',
 'CCC',
 'CAC',
 'TGG',
 'GGG',
 'CCC',
 'CAC',
 'CGC',
 'GCC',
 'CCA',
 'GCC',
 'CCC',
 'CGT',
 'GGG',
 'CCC',
 'CGC',
 'GCC',
 'AGC',
 'CCT',
 'GAC',
 'CCG',
 'GGT',
 'CTC',
 'AGC',


In [142]:
len(['TCC',
 'GGT',
 'CTC',
 'AGC',
 'AGG',
 'CCC',
 'AGG',
 'AGG',
 'GCC',
 'TCC',
 'TGG',
 'GTG',
 'GGG',
 'GGC',
 'GGG',
 'ACG',
 'CCG',
 'GGT',
 'CCC',
 'TAG'])*3

60

In [156]:
seq = ''.join([leader5[61:][i:i+3] for i in range(0, len(leader5[61:]), 3)])

len(seq)

408

In [160]:
# split seq into 2 sequences with 225 long 

len(seq[0:225]) / 3

75.0

In [159]:
len(seq[-225:]) / 3

75.0

In [161]:
part1 = flank_left + RS1 + seq[0:225] + 'GG' + RS2 + flank_right
part2 = flank_left + RS1 + seq[-225:] + 'GG' + RS2 + flank_right

len(part1), len(part2)

(283, 283)

In [162]:
part1

'GCAcATTATACCaCAACTACACAGATCTGGGCTGGTGGTCACTCAGGGTGGGGCGTGTCGCCCCTCCCCCGTCCACCTGCTCTACTCTTCCCCCGCGTGCCCTGGGCTGACCCTTGTCCCCTCCTCTCCCCGCCCCCGGTGGCAGTGGCGGCTGCTGTTGTCACCCACCGGGCCTCCTGTCCCGCTTGCCCTCCCCGCCGCGGGGCCGGCCGGGCCAGAGACAGGCGGTCGCCTTTTCAGCGCCGCCACCGCCGGGCTAGCACCATGGGTaAAACAGACTTTa'

In [163]:
part2

'GCAcATTATACCaCAACTACACAGATCTGGGCCAGAGACAGGCGGTCGCCTTTTCAGCGCCGCCACCGCCGCCATGCTGGCCGCTCGCCCACCCCACTGGGGGCCCCACCGCGCCCCAGCCCCCCGTGGGCCCCGCGCCAGCCCTGACCCGGGTCTCAGCGGCGGTGGCAGCCGAGGTGCAGGATGCAAGAAGGCGCCCCCCGGCCGGGCTCCCGCTCCAGGCCTCGCTCCCCTGCGGCCCTCTGAGCCCACCGGGCTAGCACCATGGGTaAAACAGACTTTa'

### GTF3A

In [166]:
tr_id = 'ENST00000381140.8'

tr_seq = metadata_pc_g25[metadata_pc_g25['gene'] == 'GTF3A'].iloc[0].transcript_seq

tr_seq[194:]

'CTGGATCCGCCGGCCGTGGTCGCCGAGTCGGTGTCGTCCTTGACCATCGCCGACGCGTTCATTGCAGCCGGCGAGAGCTCAGCTCCGACCCCGCCGCGCCCCGCGCTTCCCAGGAGGTTCATCTGCTCCTTCCCTGACTGCAGCGCCAATTACAGCAAAGCCTGGAAGCTTGACGCGCACCTGTGCAAGCACACGGGGGAGAGACCATTTGTTTGTGACTATGAAGGGTGTGGCAAGGCCTTCATCAGGGACTACCATCTGAGCCGCCACATTCTGACTCACACAGGAGAAAAGCCGTTTGTTTGTGCAGCCAATGGCTGTGATCAAAAATTCAACACAAAATCAAACTTGAAGAAACATTTTGAACGCAAACATGAAAATCAACAAAAACAATATATATGCAGTTTTGAAGACTGTAAGAAGACCTTTAAGAAACATCAGCAGCTGAAAATCCATCAGTGCCAGCATACCAATGAACCTCTATTCAAGTGTACCCAGGAAGGATGTGGGAAACACTTTGCATCACCCAGCAAGCTGAAACGACATGCCAAGGCCCACGAGGGCTATGTATGTCAAAAAGGATGTTCCTTTGTGGCAAAAACATGGACGGAACTTCTGAAACATGTGAGAGAAACCCATAAAGAGGAAATACTATGTGAAGTATGCCGGAAAACATTTAAACGCAAAGATTACCTTAAGCAACACATGAAAACTCATGCCCCAGAAAGGGATGTATGTCGCTGTCCAAGAGAAGGCTGTGGAAGAACCTATACAACTGTGTTTAATCTCCAAAGCCATATCCTCTCCTTCCATGAGGAAAGCCGCCCTTTTGTGTGTGAACATGCTGGCTGTGGCAAAACATTTGCAATGAAACAAAGTCTCACTAGGCATGCTGTTGTACATGATCCTGACAAGAAGAAAATGAAGCTCAAAGTCAAAAAATCTCGTGAAAAACGGAGTTTGGCCTCTCATCTCAGTGGATATATCCCTCCCAAAAGG

In [172]:
seq = tr_seq[194-120:194+105]

#[seq[i:i+3] for i in range(0, len(seq), 3)]

In [175]:
40*3, 35*3

(120, 105)

In [174]:
RS1, RS2

('AGATCT', 'GCTAGC')

In [176]:
for el in [seq[i:i+3] for i in range(0, len(seq), 3)]:
    if el == 'ATG':
        print (el)

In [177]:
gtf3a = flank_left + RS1 + seq + 'GG' + RS2 + flank_right

len(gtf3a)

283

In [178]:
gtf3a

'GCAcATTATACCaCAACTACACAGATCTGAAGTGTGCCGGCGTCGCGCGAAGGTTCAGCAGGGAGCCGTGGGCCGGGCGCGCCGGTTCCCGGCACGTGTCTCGGCACGTGGCAGCGCGCCTGGCCCTGGGCTTGGAGGCGCCGGCGCCCTGGATCCGCCGGCCGTGGTCGCCGAGTCGGTGTCGTCCTTGACCATCGCCGACGCGTTCATTGCAGCCGGCGAGAGCTCAGCTCCGACCCCGCCGCGCCCCGCGGGGCTAGCACCATGGGTaAAACAGACTTTa'

# EIF4G2

In [180]:
metadata_pc_g25[metadata_pc_g25['gene'] == 'EIF4G2'].iloc[0]

tr_id = 'ENST00000526148.5'



tr_id                                              ENST00000526148.5
gene                                                          EIF4G2
gene_tr                                                   EIF4G2-006
transcript_seq     GGGAGGAATGCTCCGCTCTTTTGTTCCCAGGAAGGGAAAAACGGGG...
5UTR_start_seq     GGGAGGAATGCTCCGCTCTTTTGTTCCCAGGAAGGGAAAAACGGGG...
CDS_start                                                        AAA
cds_seq            GTGGAGAGTGCGATTGCAGAAGGGGGTGCTTCTCGTTTCAGTGCTT...
cds_start_codon                                                  GTG
cds_stop_codon                                                   TAA
cds_start_pos                                                    511
cds_stop_pos                                                    3235
utr5_start                                                         0
utr5_end                                                         511
record_id          ENST00000526148.5|ENSG00000110321.15|OTTHUMG00...
Name: 46415, dtype: object

In [193]:
seq = metadata_pc_g25[metadata_pc_g25['gene'] == 'EIF4G2'].iloc[0].transcript_seq[511:]

seq[0:6]

'GTGGAG'

In [188]:
50*3, 25*3

(150, 75)

In [189]:
511-50*3, 511+25*3

(361, 586)

In [190]:
SEQ = metadata_pc_g25[metadata_pc_g25['gene'] == 'EIF4G2'].iloc[0].transcript_seq[361:586]

len(SEQ)

225

In [194]:
for el in [SEQ[i:i+3] for i in range(0, len(SEQ), 3)]:
    if el == 'ATG':
        print (el)

In [195]:
eif4g2 = flank_left + RS1 + SEQ + 'GG' + RS2 + flank_right


len(eif4g2)

283

In [196]:
eif4g2

'GCAcATTATACCaCAACTACACAGATCTCGTTTAAAATGCTCTTGTGTTCTACGAAGTCTCTTTGCATCCTTGAACACCGGGGAATTTCCAGGTATTTCATTTCTCCTGTCCCCTCCCCTCCCCACCCCATCTATTAATATTATTCTTTTGAAGATTCTTCGTTGTCAAGCCGCCAAAGTGGAGAGTGCGATTGCAGAAGGGGGTGCTTCTCGTTTCAGTGCTTCTTCGGGCGGAGGAGGAAGTAGGGGTGCAGGGCTAGCACCATGGGTaAAACAGACTTTa'

# Final check: 
* new in-frame ATG-CTG-GTG (avoid!)
* new out-of-frame ATG, CTG (avoid!)
* no in-frame stops introduced 

In [356]:
li_clean = []

for el in pset_test_cassettes3[['tr_id', 'gene', 'coo', 
                                'no_RS', 'in_frame_ATG_in_theoretical_extension', 
                               'test_cassette_seq']].to_numpy():
    
    tr_id = el[0]
    gene = el[1]
    coo = el[2]
    seq = el[3]
    atg = el[4]
    orig = el[5]
    
    # check for RS again  
    if (seq.find(RS1) != -1) | (seq.find(RS1) != -1):
        print (tr_id, gene, coo, seq)
        
    # check for in-frame stops
    # TAA in ATC-rich sequence 
    atc = pset_test_cassettes3[(pset_test_cassettes3['tr_id'] == tr_id) & 
                               (pset_test_cassettes3['coo'] == coo)].iloc[0].test_cassette_seq_.split('TAA')
    triplets = [seq[i:i+3] for i in range(0, len(seq), 3)]
    counts= 0
    for codon in triplets:
        if (codon == 'TGA') | (codon == 'TAA') | (codon == 'TAG'):
            #print (tr_id, gene, coo, seq, codon, seq.find(codon))
            counts += 1
    
    counts2 = 0
    triplets2 = [atc[i:i+3] for i in range(0, len(atc), 3)]        
    for codon in triplets:
        if (codon == 'TGA') | (codon == 'TAA') | (codon == 'TAG'):
            counts2 += 1
    
    
    # ALL TAAs in ATC-rich sequences 
    if counts != counts2: 
        print (tr_id, gene, counts, counts2)  
        
        
        
    # check for in-frame and out-of-frame AUGs in orig and final sequences 
    for i in range(0, len(seq), 1):
        final_triplet = seq[i:i+3]
        orig_triplet = orig[i:i+3]
        frame = i % 3 
        
        #if final_triplet != orig_triplet:
            #print (tr_id, gene, orig_triplet, final_triplet, frame)
        
        # if orig triplet is not ATG, but the final seq has introduced ATG
        if (final_triplet == 'ATG') & (orig_triplet != 'ATG'):
            print (tr_id, gene, orig_triplet, final_triplet, frame)
            
        # if orig triplet is not CTG, but the final seq has introduced GTG
        if (final_triplet == 'CTG') & (orig_triplet != 'CTG'):
            print (tr_id, gene, orig_triplet, final_triplet, frame)
            
        # if orig triplet is not GTG, but the final seq has introduced GTG
        if (final_triplet == 'GTG') & (orig_triplet != 'GTG'):
            print (tr_id, gene, orig_triplet, final_triplet, frame)
            
        
    # check for in-frame AUGs (new ones)
    # no new ones are there 
    #for i, codon in enumerate(triplets):
    #    if codon == 'ATG':
    #        print (tr_id, gene, coo, seq, atg)
    #        print ('pos ATG is %s' % (i*3))
    #        print ()
            
    # check for out-of-frame AUGs and CUGs 
    #for i in range(0, len(seq), 1):
        #sliced = seq[i:i+3]
        #frame = i % 3 # 0/3 = 0, 
        #print (sliced)
        #if (sliced == 'ATG') & (frame != 0) & ():
            

ENST00000259154.8 KCTD3 GCG GTG 1
ENST00000259154.8 KCTD3 GCG GTG 1
ENST00000436587.6 C1GALT1 GCG GTG 1
ENST00000265462.8 PRDX5 GCG GTG 1


# Add GG at the end, RS1, RS2, flanks (15nt)

In [365]:
len(flank_left)

15

In [366]:
len(flank_right)

15

In [367]:
flank_left = 'GCAcATTATACCaCAACTACAC'
flank_right = 'ACCATGGGTaAAACAGACTTTa'

In [368]:
len(flank_left)

22

In [369]:
len(flank_right)

22

In [370]:
pre_final_df['269nt_cassette_longer_flanks'] = [flank_left+RS1+x+'GG'+RS2+flank_right for x in pre_final_df['no_RS']]

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [371]:
pre_final_df['len_longer_flanks'] = pre_final_df['269nt_cassette_longer_flanks'].str.len()

pre_final_df[pre_final_df['len'] == 269]

,tr_id,gene,nte_seq,coo,type,set,in_frame_ATG_in_theoretical_extension,mark,no_RS,269nt_cassette,len,269nt_cassette_longer_flanks,len_longer_flanks
0,ENST00000338370.7,AURKAIP1,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,"[0, 225]",long,RiboSET,,-,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,ACCATGGGTaAAACAAGATCTAGGTCGGGCTTGGGAAGGGTCAGCG...,269,GCAcATTATACCaCAACTACACAGATCTAGGTCGGGCTTGGGAAGG...,283
1,ENST00000338370.7,AURKAIP1,AGGTCGGGCTTGGGAAGGGTCAGCGGGAGGCCTGAGGGCGCCGGGC...,"[174, 399]",long,RiboSET,,-,ACACCGCCCGCGTCGGGGGCTTGGGAGCCTCGGGGCGCGAGCTGTG...,ACCATGGGTaAAACAAGATCTACACCGCCCGCGTCGGGGGCTTGGG...,269,GCAcATTATACCaCAACTACACAGATCTACACCGCCCGCGTCGGGG...,283
2,ENST00000455833.6,IFFO2,CAGAAGGGGCAGTGCCGGCCGAGAGTCCTCTGCGTGCGGCTGCCGC...,"[0, 225]",long,PhyloSET,,-,CAGAAGGGGCAGTGCCGGCCGAGAGTCCTCTGCGTGCGGCTGCCGC...,ACCATGGGTaAAACAAGATCTCAGAAGGGGCAGTGCCGGCCGAGAG...,269,GCAcATTATACCaCAACTACACAGATCTCAGAAGGGGCAGTGCCGG...,283
3,ENST00000455833.6,IFFO2,CAGAAGGGGCAGTGCCGGCCGAGAGTCCTCTGCGTGCGGCTGCCGC...,"[129, 354]",long,PhyloSET,,-,CGTCCCGAGGAGCGGCCGAGTGCCCTCTCTCGCCTCGGCGCCCGCA...,ACCATGGGTaAAACAAGATCTCGTCCCGAGGAGCGGCCGAGTGCCC...,269,GCAcATTATACCaCAACTACACAGATCTCGTCCCGAGGAGCGGCCG...,283
4,ENST00000302118.5,PCSK9,CGACGTCGAGGCGCTCATGGTTGCAGGCGGGCGCCGCCGTTCAGTT...,"[0, 225]",long,RiboSET,,-,CGACGTCGAGGCGCTCATGGTTGCAGGCGGGCGCCGCCGTTCAGTT...,ACCATGGGTaAAACAAGATCTCGACGTCGAGGCGCTCATGGTTGCA...,269,GCAcATTATACCaCAACTACACAGATCTCGACGTCGAGGCGCTCAT...,283
...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,ENST00000381192.8_PAR_Y,CD99,GCGCGTGGAGGCCGGGGCGGGGCGGGCGCAGCCGGCGCTGAGCTTG...,[45],short,RiboSET,,-,AAATCTAATTAAATCTCTACTAAATTTACCCTTCACTCTACTCCAG...,ACCATGGGTaAAACAAGATCTAAATCTAATTAAATCTCTACTAAAT...,269,GCAcATTATACCaCAACTACACAGATCTAAATCTAATTAAATCTCT...,283
606,ENST00000298585.2,ZMYND19,GTGGCGGTGGCGGCGGCGGCGGCGGCGCGTGGCAGGTCCGGCGGGC...,[225],exact,RiboSET,,-,GTGGCGGTGGCGGCGGCGGCGGCGGCGCGTGGCAGGTCCGGCGGGC...,ACCATGGGTaAAACAAGATCTGTGGCGGTGGCGGCGGCGGCGGCGG...,269,GCAcATTATACCaCAACTACACAGATCTGTGGCGGTGGCGGCGGCG...,283
607,ENST00000245539.10,MRPS7,AGTGAGTGCCGTCACCGAGGGCCGCGCCAGACTGCGACGGATACAG...,[225],exact,RiboSET,,-,AGTGAGTGCCGTCACCGAGGGCCGCGCCAGACTGCGACGGATACAG...,ACCATGGGTaAAACAAGATCTAGTGAGTGCCGTCACCGAGGGCCGC...,269,GCAcATTATACCaCAACTACACAGATCTAGTGAGTGCCGTCACCGA...,283
608,ENST00000299608.6,TMX3,CTGCGCAGGCGCTGTCGCATGCGCCGCACTCCCCTCGCATGCGCCG...,[225],exact,RiboSET,ATG in 18 position in ext;,RS_removed,CTGCGCAGGCGCTGTCGCATGCGCCGCACTCCCCTCGCATGCGCCG...,ACCATGGGTaAAACAAGATCTCTGCGCAGGCGCTGTCGCATGCGCC...,269,GCAcATTATACCaCAACTACACAGATCTCTGCGCAGGCGCTGTCGC...,283


In [372]:
pre_final_df.to_csv('data/experimental_cassettes_with_RS_2nt_and_22nt_flanks.txt', sep='\t', index=False)

# Check in-frame starts for known sequences

In [360]:
pre_final_df[pre_final_df['gene'] == 'CCDC8'].iloc[0]['269nt_cassette']

'ACCATGGGTaAAACAAGATCTTTTGTTGGGGCTCATTGCCCCCTCACCCCAGATATCACCCTGGAGATTTTAAAGACTCTCGAGAAAAGCCACGTGGGGGGCTGGTTCCCCTGGGGCTTCCTGCCGTCCCCCGACTGCCTCATTCTTTGGAGCGTCCCCGATGTCTGCAAAGATGTGGATTTGGACGTCCTCGTGGAAGCCCTAAAGCCCGTGGGGACATTTAAGAAGATCGGCAAGGTGTTCCGCGGGCTAGCATACCaCAACTACAC'

In [361]:
len('ACCATGGGTaAAACAAGATCTTTTGTTGGGGCTCATTGCCCCCTCACCCCAGATATCACCCTG') % 3

0

In [362]:
rset[rset['gene'] == 'CCDC8']

,tr_id,gene,N_term_start1,N_term_end1,len_codons,global_coo_50_and_less,strand,global_coo_primary,PhyloCSF120score,number_of_records,ovlp,Riboseq_Summary,Coverage_value_ext,Proteomics_count_ext,tag,transcript_type,CDS_ratio
346,ENST00000307522.3,CCDC8,525,774,83.0,chr19:46412811-46412960,-,chr19:46412811-46413059,2693.8893,108,0;0;0,ATC; 559-775; Rank: 400; cov: 79.1,79.104478,7.666008,"basic,appris_principal_1,CCDS",protein_coding,1.00811


In [363]:
pre_final_df[pre_final_df['gene'] == 'CCDC8'].iloc[1]['269nt_cassette']

'ACCATGGGTaAAACAAGATCTACCCCAGATATCACCCTGGAGATTTTAAAGACTCTCGAGAAAAGCCACGTGGGGGGCTGGTTCCCCTGGGGCTTCCTGCCGTCCCCCGACTGCCTCATTCTTTGGAGCGTCCCCGATGTCTGCAAAGATGTGGATTTGGACGTCCTCGTGGAAGCCCTAAAGCCCGTGGGGACATTTAAGAAGATCGGCAAGGTGTTCCGCAAGGAGGAGGACTCCACGGTGGGGGGGCTAGCATACCaCAACTACAC'

In [364]:
len('ACCATGGGTaAAACAAGATCTACCCCAGATATCACCCTG') / 3

13.0

In [379]:
'GCAcATTATACCaCAACTACACAGATCTTTTGTTGGGGCTCATTGCCCCCTCACCCCAGATATCACCCTGGAGATTTTAAAGACTCTCGAGAAAAGCCACGTGGGGGGCTGGTTCCCCTGGGGCTTCCTGCCGTCCCCCGACTGCCTCATTCTTTGGAGCGTCCCCGATGTCTGCAAAGATGTGGATTTGGACGTCCTCGTGGAAGCCCTAAAGCCCGTGGGGACATTTAAGAAGATCGGCAAGGTGTTCCGCGGGCTAGCACCATGGGTaAAACAGACTTTa'

'GCAcATTATACCaCAACTACACAGATCTTTTGTTGGGGCTCATTGCCCCCTCACCCCAGATATCACCCTGGAGATTTTAAAGACTCTCGAGAAAAGCCACGTGGGGGGCTGGTTCCCCTGGGGCTTCCTGCCGTCCCCCGACTGCCTCATTCTTTGGAGCGTCCCCGATGTCTGCAAAGATGTGGATTTGGACGTCCTCGTGGAAGCCCTAAAGCCCGTGGGGACATTTAAGAAGATCGGCAAGGTGTTCCGCGGGCTAGCACCATGGGTaAAACAGACTTTa'